# Exploring surface mass balance time series 

Welcome!  This notebook is intended to guide you through the preliminary data processing and visualization that helps us choose candidate models to fit to the Greenland surface mass balance time series.

To begin with, we have aggregated gridded, Greenland-wide surface mass balance from GrSMBMIP participating models into catchment-integrated time series, based on the catchment boundaries defined by [Mouginot & Rignot](https://datadryad.org/stash/dataset/doi:10.7280/D1WT11).  The code for aggregation is in `multi-catchment-process.py` in this repository.

With this data, we will plot catchment-aggregated time series and explore some relevant statistics.

### Preliminaries - imports, names, paths

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from matplotlib import cm
import glob

In [ ]:
data_directory = '/Users/lizz/Documents/GitHub/stoch-SMB/SMBMIP-processed/'
model_names = ['ANICE-ITM_Berends', 'CESM_kampenhout', 'dEBM_krebs','HIRHAM_mottram', 
                'NHM-SMAP_niwano', 'RACMO_noel', 'SNOWMODEL_liston']

In [ ]:
example_basin = 101 # an ID number assigned by Mouginot & Rignot, between 1-200.
## Some popular choices: 0 for Umiammakku Isbrae, 15 for Kangiata Nunaata Sermia, 
## 62 for Helheim Glacier, 101 for Kangerlussuaq Glacier 

ctmt_fpath = glob.glob(data_directory+'*-catchment_{}-tseries.csv'.format(example_basin))[0]

## Read in and plot monthly time series

In [ ]:
def read_catchment_series(fpath, anomaly=False):
    catchment_fpath = fpath
    catchment_tseries = pd.read_csv(catchment_fpath, index_col=0, parse_dates=[0])
    catchment_tseries.mask(catchment_tseries>1e30)
    anomaly_series = catchment_tseries - catchment_tseries.mean()
    if anomaly:
        return anomaly_series
    else:
        return catchment_tseries